# Importing necessary libraries

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np

import pm4py as pm4

import plotly.express as px

import pickle

from xgboost import XGBRegressor
from hyperopt import hp, tpe, Trials, fmin, space_eval, STATUS_OK
from Split_functions import data_split

## Loading the data

In [ ]:
df = pd.read_csv('cleaned_data.csv')
df.head()

## Feature engineering

#### Retrieving month/day/weekday/hour and if its a holiday or not

In [ ]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format = 'mixed')

In [ ]:
df['hour'] = df['time:timestamp'].dt.hour
df['day'] = df['time:timestamp'].dt.day
df['month'] = df['time:timestamp'].dt.month
df['weekday'] = df['time:timestamp'].dt.strftime("%A")
df['is_holiday'] = 0

#### Creating new column working hour

In [ ]:
work_hours = df.groupby('hour').count()
work_hours['percentage'] = work_hours['concept:name'].apply(lambda x : x/sum(work_hours['concept:name'])*100)
work_hours_list = work_hours[work_hours['percentage']>1].reset_index()['hour'].to_list()

In [ ]:
# Determining if it is the working hours or not
df['work_hour'] = df['hour'].apply(lambda x: 1 if x in(work_hours_list) else 0)

#### Adding holidays 

In [ ]:
# Typical weekends
df.loc[(df['weekday'] == 'Sunday') | (df['weekday'] == 'Saturday'), 'is_holiday'] = 1

# New Year's Day
df.loc[(df['day'] == 1) & (df['month'] == 1), 'is_holiday'] = 1

# Christmas Day 
df.loc[((df['day'].isin([i for i in range(22, 27)]))) & (df['month'] == 1), 'is_holiday'] = 1

# Good Friday, Easter 
df.loc[(df['day'].isin([i for i in range(6,10)])) & (df['month'] == 4), 'is_holiday'] = 1

# King's day (27 April)
df.loc[(df['day'] == 27) & (df['month'] == 4), 'is_holiday'] = 1

# Liberation Day
df.loc[(df['day'] == 5) & (df['month'] == 5), 'is_holiday'] = 1

# Ascension Day 
df.loc[(df['day'].isin([i for i in range(17, 21)])) & (df['month'] == 5), 'is_holiday'] = 1

# Pentecost
df.loc[(df['day'].isin([i for i in range(26, 29)])) & (df['month'] == 5), 'is_holiday'] = 1

#### Retrieving time delta between events and applying logarithmic normalization

In [ ]:
df['current_time_delta'] = df.groupby('case:concept:name')['time:timestamp'].diff(-1).dt.total_seconds().abs()
df['logged_current_time_delta'] = np.log(df['current_time_delta'] + 1)

In [ ]:
clown = df.groupby('case:concept:name').count().sort_values(by = 'concept:name')
needed_ids = clown[clown['concept:name'] <= 82].reset_index()['case:concept:name'].unique()

In [ ]:
df = df[df['case:concept:name'].isin(needed_ids)]

#### Retrieving time lags of previous events and previous activities

In [ ]:
df['next_activity_time'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(-1)
df['lag1'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(1)
df['lag2'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(2)
df['lag3'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(3)
df['lag4'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(4)
df['lag5'] = df.groupby('case:concept:name')['logged_current_time_delta'].shift(5)

df['previous_activity1'] = df.groupby('case:concept:name')['concept:name'].shift(1)
df['previous_activity2'] = df.groupby('case:concept:name')['concept:name'].shift(2)
df['previous_activity3'] = df.groupby('case:concept:name')['concept:name'].shift(3)
df['previous_activity4'] = df.groupby('case:concept:name')['concept:name'].shift(4)
df['previous_activity5'] = df.groupby('case:concept:name')['concept:name'].shift(5)


# df = pd.get_dummies(df, columns=['case:concept:name', 'previous_activity1', 'previous_activity2', 'previous_activity3', 'previous_activity4','previous_activity5'], dtype = int)
le = LabelEncoder()
df['current_activity_encoded'] = le.fit_transform(df['concept:name'])
df['previous_activity1_encoded'] = le.fit_transform(df['previous_activity1'])
df['previous_activity2_encoded'] = le.fit_transform(df['previous_activity2'])
df['previous_activity3_encoded'] = le.fit_transform(df['previous_activity3'])
df['previous_activity4_encoded'] = le.fit_transform(df['previous_activity4'])
df['previous_activity5_encoded'] = le.fit_transform(df['previous_activity5'])

In [ ]:
df

In [ ]:
# df = pd.get_dummies(df, columns=['concept:name', 'previous_activity1', 'previous_activity2', 'previous_activity3', 'previous_activity4','previous_activity5'], dtype = int)

In [ ]:
df = df.fillna(1e-6)

### Splitting the data on train and test sets

In [ ]:
predictor = df[['logged_current_time_delta', 
                'current_activity_encoded', 'previous_activity1_encoded', 'previous_activity2_encoded', 
                'previous_activity3_encoded', 'previous_activity4_encoded', 'previous_activity5_encoded',
                'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
                'work_hour', 'is_holiday', 'month', 'case:concept:name', 'time:timestamp']]
target = df[['next_activity_time', 'case:concept:name', 'time:timestamp']]
train_size = 0.8

X, X_test, y, y_test = data_split(predictor, target, train_size)

print('+----------------------------------------------------------------+')
print('After cleaning traces!')
print('Training dataset max time:',X['time:timestamp'].max())
print('Testing dataset min time:', X_test['time:timestamp'].min())
print('+----------------------------------------------------------------+')

In [ ]:
X_features = ['logged_current_time_delta', 
                'current_activity_encoded', 'previous_activity1_encoded', 'previous_activity2_encoded', 
                'previous_activity3_encoded', 'previous_activity4_encoded', 'previous_activity5_encoded',
                'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
                'work_hour', 'is_holiday', 'month', 'case:concept:name']

y_features = ['next_activity_time', 'case:concept:name']

In [ ]:
X_features_to_train = ['logged_current_time_delta', 
                'current_activity_encoded', 'previous_activity1_encoded', 'previous_activity2_encoded', 
                'previous_activity3_encoded', 'previous_activity4_encoded', 'previous_activity5_encoded',
                'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
                'work_hour', 'is_holiday', 'month']

y_features_to_train = ['next_activity_time']

In [ ]:
X = X[X_features]
X_test = X_test[X_features]
y = y[y_features]
y_test = y_test[y_features]

In [ ]:
X = X.reset_index(drop = True)
y = y.reset_index(drop = True)

In [ ]:
# showcase a distribution of the time till next activity
# put np.log on the time till next activity in order to get a better distribution
# one hot encoding for features, as well put their order in the trace
# put instead of nan values the 1e6 (just a really small value)

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(df['current_time_delta'], opacity = 0.5)

In [ ]:
px.scatter(df['logged_current_time_delta'], opacity = 0.2)

# Random Forest Regression

In [ ]:
# Define the search space for hyperparameters
space = {
    'n_estimators': hp.choice('n_estimators', [int(x) for x in np.linspace(start = 5, stop = 100, num = 60)]),
    'max_depth': hp.choice('max_depth', [5, 6, 7, 9, 10, 12, 13, 15, 16, 17, 19, 20, 22, 23, 25]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [2, 4, 6, 8])
}

# Initialize variables to store results
best_params_rfr_list = []
best_scores_list = []

n = 10

start = 0
end = len(X)
step_size = end//n

train_start = 0 
train_end = end - step_size

test_start = train_end
test_end = end

for i in range(n):
    if train_start == test_start:
        train_x = X.loc[test_end+1:]
        train_y = y.loc[test_end+1:]

        test_x = X.loc[test_start:test_end]
        test_y = y.loc[test_start:test_end]

    else:
        if test_end + 1 >= len(X):
            train_x = X.loc[train_start:train_end-1]
            train_y = y.loc[train_start:train_end-1]
        else:
            train_x = pd.concat([X.loc[train_start:train_end-1], X.loc[test_end+1:]])
            train_y = pd.concat([y.loc[train_start:train_end-1], y.loc[test_end+1:]])

        test_x = X.loc[test_start:test_end]
        test_y = y.loc[test_start:test_end]
    
    overlapping_sets = list(set(train_x['case:concept:name'].unique()).intersection(set(test_x['case:concept:name'].unique())))
    # # Clean train
    X_train = train_x[train_x['case:concept:name'].isin([overlapping_sets]) == False]
    y_train = train_y[train_y['case:concept:name'].isin(train_x['case:concept:name'].unique())]
    
    # # Clean test
    X_validation = test_x[test_x['case:concept:name'].isin([overlapping_sets]) == False]
    y_validation = test_y[test_y['case:concept:name'].isin(test_x['case:concept:name'].unique())]

    # # Finalizing the data
    X_train = X_train[X_features[:-1]].values
    X_validation = X_validation[X_features[:-1]].values
    y_train = y_train[y_features[0]].values
    y_validation = y_validation[y_features[0]].values
    
    # Define a function to optimize using Hyperopt
    def objective(params):
        xgb = RandomForestRegressor(**params, n_jobs = -1)
        xgb.fit(X_train, np.ravel(y_train))
        score = xgb.score(X_validation, y_validation)
        return {'loss': -score, 'status': STATUS_OK}
    
    # Define Trials object to store optimization results
    trials = Trials()
    
    # Use Hyperopt to find the best hyperparameters
    best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials, return_argmin=False)
    
    # Store the best parameters and corresponding score
    best_params_rfr_list.append(best)
    best_scores_list.append(-trials.best_trial['result']['loss'])  # Convert back to positive
    
    
    test_end = test_start
    train_end -= step_size
    test_start = train_end

#Print the best parameters and average score across all outer folds
print("Best Parameters:")
for params in best_params_rfr_list:
    print(params)
print("Average Score:", np.mean(best_scores_list))

In [ ]:
# saving time models in pickle format
count = 1
for i in best_params_rfr_list:
    model = RandomForestRegressor(**i)
    
    model.fit(X[X_features_to_train].values, np.ravel(y[y_features_to_train].values))
    print(model.score(X_test[X_features_to_train].values, y_test[y_features_to_train].values))
    print(f'MAE: {round(mean_absolute_error(np.exp(y_test[y_features_to_train].values), np.exp(model.predict(X_test[X_features_to_train].values)))/3600,3)} hours')
    
    pickle.dump(model , open(f'next_activity_time_prediction_rfr_{count}.pk1' , 'wb'))
    count+=1

# XGBoost Regression

In [ ]:
# Define the search space for hyperparameters
space = {
    'n_estimators': hp.choice('n_estimators', [int(x) for x in np.linspace(start = 5, stop = 50, num = 45)]),
    'max_depth': hp.choice('max_depth', [int(i) for i in range(2,20)]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.6),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.2),
}

# Initialize variables to store results
best_params_xgbr_list = []
best_scores_list = []

n = 10

start = 0
end = len(X)
step_size = end//n

train_start = 0 
train_end = end - step_size

test_start = train_end
test_end = end

for i in range(n):
    if train_start == test_start:
        train_x = X.loc[test_end+1:]
        train_y = y.loc[test_end+1:]

        test_x = X.loc[test_start:test_end]
        test_y = y.loc[test_start:test_end]

    else:
        if test_end + 1 >= len(X):
            train_x = X.loc[train_start:train_end-1]
            train_y = y.loc[train_start:train_end-1]
        else:
            train_x = pd.concat([X.loc[train_start:train_end-1], X.loc[test_end+1:]])
            train_y = pd.concat([y.loc[train_start:train_end-1], y.loc[test_end+1:]])

        test_x = X.loc[test_start:test_end]
        test_y = y.loc[test_start:test_end]
    
    overlapping_sets = list(set(train_x['case:concept:name'].unique()).intersection(set(test_x['case:concept:name'].unique())))
    # # Clean train
    X_train = train_x[train_x['case:concept:name'].isin([overlapping_sets]) == False]
    y_train = train_y[train_y['case:concept:name'].isin(train_x['case:concept:name'].unique())]
    
    # # Clean test
    X_validation = test_x[test_x['case:concept:name'].isin([overlapping_sets]) == False]
    y_validation = test_y[test_y['case:concept:name'].isin(test_x['case:concept:name'].unique())]

    # # Finalizing the data
    X_train = X_train[X_features[:-1]].values
    X_validation = X_validation[X_features[:-1]].values
    y_train = y_train[y_features[0]].values
    y_validation = y_validation[y_features[0]].values
    

    # Define a function to optimize using Hyperopt
    def objective(params):
        xgb = XGBRegressor(**params, n_jobs = -1)
        xgb.fit(X_train, y_train)
        score = xgb.score(X_validation, y_validation)
        return {'loss': -score, 'status': STATUS_OK}
    
    # Define Trials object to store optimization results
    trials = Trials()
    
    # Use Hyperopt to find the best hyperparameters
    best = fmin(objective, space, algo=tpe.suggest, max_evals=20, trials=trials, return_argmin=False)
    
    # Store the best parameters and corresponding score
    best_params_xgbr_list.append(best)
    best_scores_list.append(-trials.best_trial['result']['loss'])  # Convert back to positive
    
    
    test_end = test_start
    train_end -= step_size
    test_start = train_end

#Print the best parameters and average score across all outer folds
print("Best Parameters:")
for params in best_params_xgbr_list:
    print(params)
print("Average Score:", np.mean(best_scores_list))

In [ ]:
# saving time models in pickle format
count = 1
for i in best_params_xgbr_list:
    model = XGBRegressor(**i)
    model.fit(X[X_features_to_train].values, np.ravel(y[y_features_to_train].values))
    print(model.score(X_test[X_features_to_train].values, y_test[y_features_to_train].values))

    print(f'MAE on logged values (used to get a better smoothing): {round(mean_absolute_error(np.exp(y_test[y_features_to_train].values), np.exp(model.predict(X_test[X_features_to_train].values)))/3600,3)} hours')
    pickle.dump(model, open(f'next_activity_time_prediction_xgbr_{count}.pk1', 'wb'))
    count += 1